In [1]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#   Modelos lineales y modelos lineales generalizados
# ----------------------------------------------------------
#        Rolando Gonzales Martinez, Julio 2024
# ==========================================================
# Modelo lineal multivariante: exogeneidad 

# Importando librerias:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms


# Cargar el conjunto de datos:
url = 'https://raw.githubusercontent.com/rogon666/UMSA/main/MLMLG/datos/salarios.csv'

# Cargar los datos en un DataFrame
datos = pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
print(datos.head())

   salario  educacion  educacionmadre  edad  antiguedad_ejecutivo  \
0     1161         12              11    49                     6   
1      600         10               9    43                     5   
2      379          9               8    51                     5   
3      651          8               9    55                     4   
4      497         10               9    44                     5   

   educacion_grado  antiguedad  educacion_posgrado  valor_empresa  \
0                1           9                   1          23200   
1                1          10                   1           1100   
2                1           9                   1           1100   
3                1          22                   0           1000   
4                1           8                   1            387   

   beneficios_empresa  ventas_empresa  
0                 966            6200  
1                  48             283  
2                  40             169  
3         

In [2]:
Xeduc = datos['educacion']
Xposg = datos['edad']
Xante = datos['antiguedad_ejecutivo']
ylogs = np.log(datos['salario'])

X = np.column_stack((Xeduc, Xposg, Xante))
X = pd.DataFrame(X, columns=['educacion', 'edad', 'antiguedad'])
X = sm.add_constant(X)
X.columns = ['constante', 'educacion', 'edad', 'antiguedad'] 
modelo_mco = sm.OLS(ylogs,X).fit()
print(modelo_mco.summary())

                            OLS Regression Results                            
Dep. Variable:                salario   R-squared:                       0.645
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                     104.8
Date:                Wed, 24 Jul 2024   Prob (F-statistic):           1.04e-38
Time:                        18:12:38   Log-Likelihood:                -70.342
No. Observations:                 177   AIC:                             148.7
Df Residuals:                     173   BIC:                             161.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constante      4.0309      0.224     18.028      0.0

In [3]:
# Evaluación del instrumento
Zedum = datos['educacionmadre'] 
Z = sm.add_constant(Zedum)
modelo_mco_paso1 = sm.OLS(Xeduc, Z).fit()
print(modelo_mco_paso1.summary())
educacion_ajustada = modelo_mco_paso1.fittedvalues
residuos_mco_paso1 = modelo_mco_paso1.fittedvalues

                            OLS Regression Results                            
Dep. Variable:              educacion   R-squared:                       0.816
Model:                            OLS   Adj. R-squared:                  0.815
Method:                 Least Squares   F-statistic:                     777.2
Date:                Wed, 24 Jul 2024   Prob (F-statistic):           2.82e-66
Time:                        18:12:44   Log-Likelihood:                -228.04
No. Observations:                 177   AIC:                             460.1
Df Residuals:                     175   BIC:                             466.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              1.2883      0.322      3.

In [4]:
# Modelo VI (segundo paso de MCO 2 pasos)
Z = educacion_ajustada
ZX = np.column_stack((Z, Xposg, Xante))
ZX = pd.DataFrame(ZX, columns=['educacion', 'edad', 'antiguedad'])
ZX = sm.add_constant(ZX)
ZX.columns = ['constante', 'educacion', 'edad', 'antiguedad'] 
modelo_VI = sm.OLS(ylogs,ZX).fit()
print(modelo_VI.summary())

                            OLS Regression Results                            
Dep. Variable:                salario   R-squared:                       0.613
Model:                            OLS   Adj. R-squared:                  0.606
Method:                 Least Squares   F-statistic:                     91.35
Date:                Wed, 24 Jul 2024   Prob (F-statistic):           1.80e-35
Time:                        18:12:51   Log-Likelihood:                -77.995
No. Observations:                 177   AIC:                             164.0
Df Residuals:                     173   BIC:                             176.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constante      3.8888      0.243     15.973      0.0

In [5]:
import numpy as np
from scipy import stats
# Funcion para calcular el test de Hausman
def test_hausman_test(mco, vi):
    b = mco.params
    B = vi.params
    v_b = mco.cov_params()
    v_B = vi.cov_params()
    df = len(b)
    diff = b - B
    stat = np.dot(np.dot(diff.T, np.linalg.inv(v_b - v_B)), diff)
    pval = stats.chi2.sf(stat, df)
    return stat, pval

# Testeando exogeneidad:
hausman_stat, hausman_pval = test_hausman_test(modelo_mco, modelo_VI)
print(f'Estadígrafo de Hausman: {hausman_stat}')
print(f'Valor p de Hausman: {hausman_pval}')

if hausman_pval < 0.05:
    print("Rechazamos la hipótesis nula de exogeneidad. Existe evidencia de endogeneidad.")
else:
    print("No rechazamos la hipótesis nula de exogeneidad. No hay evidencia de endogeneidad.")


Estadígrafo de Hausman: 20.71947743796963
Valor p de Hausman: 0.00035990757123605295
Rechazamos la hipótesis nula de exogeneidad. Existe evidencia de endogeneidad.
